In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from glow import lenses, time_domain_c, freq_domain_c, tools

# Overview

The core of this module is written in C, it can be used in three ways:
  - Directly from C: the syntax, types of 'objects' etc. are similar to the Python version. There are examples in wrapper/glow_lib/tests (not proper examples, since this are just auxiliary test files) 
  - Directly from Python: the wrapper can be used independently from any other of our python modules (see Testing performance section below). However it is not fully functional since even though most of the algorithms are C based, they are still interfaced through Python.
  - **(Preferred)** Through `time_domain_c.py` and `freq_domain_c.py`: the new objects included here can be used in the same way as the ones in `time_domain.py` and `freq_domain.py`, i.e. they can be interfaced with `lenses.py`. **However** keep in mind that a change in the lenses will either not have any effect or (hopefully) the code will break down. The implementation of the lenses in C and python are independent and the wrapper only translates the physical parameters from Python to C.
 
Finally, the two sets (`time_domain.py`, `freq_domain.py`) and (`time_domain_c.py`, `freq_domain_c.py`) are totally independent of each other. The first one is written in full Python, while the second one relies on compiled code (C and Cython). Fron now on I will be maintaining only the latter, occasionally updating the first but without taking any effort (and time) in optimizing it further. However, I will try to keep it functional and avoid breaking compatibility with `lenses.py`.

# Time domain

## Single contour

This object performs the exact computation of $I(\tau)$ for an arbitrary lens assuming that there is only one critical point in the Fermat potential (i.e. the minimum). The algorithm tries to find the first two critical points starting from the origin and infinity, and will not proceed if they do not coincide.

We can use it as `It_WL` or any other `ItGeneral` object. We first choose a lens and then define the physical and precision parameters

In [ ]:
Psi = lenses.Psi_CIS()

y = 1.4
p_prec = {'tmin' : 1e-3,\
          'tmax' : 100,\
          'Nt' : 5000}

It = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

We can then work with it as before, either accessing its grid points (`It.t_grid`, `It.It_grid`) or use its interpolation function

In [ ]:
taus = np.geomspace(1e-1, 10, 10000)

fig, ax = plt.subplots()

ax.plot(taus, It.eval_It(taus)/2/np.pi, label=It.lens.p_phys['name'])

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.set_xscale('log')
ax.legend(loc='best');

There are two new options for this object. First one is the computation in parallel over `numpy` arrays. It is switch on by default (`parallel = True`), but it can be changed if needed.

In [ ]:
It.display_info()

The second new option is the `eval_mode = exact` (by default `eval_mode = interpolate`, which is the behaviour of the python module). In this case the grid is not computed and `eval_It` computes the integral at any point requested without using interpolation

In [ ]:
Psi = lenses.Psi_CIS()

y = 1.4
p_prec = {'eval_mode' : 'exact'}

It = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

## ---------------------------------------------------

taus = np.geomspace(1e-1, 10, 10000)
Its = It.eval_It(taus)/2/np.pi

fig, ax = plt.subplots()

ax.plot(taus, Its, label=It.lens.p_phys['name'])

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.set_xscale('log')
ax.legend(loc='best');

We can also compute the Green function $G(\tau)$

In [ ]:
Psi = lenses.Psi_CIS()

y = 1.4
p_prec = {'eval_mode' : 'exact'}

It = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

## ---------------------------------------------------

taus = np.geomspace(1e-1, 10, 1000)
Gts = It.eval_Gt(taus, dt=1e-3)  # default value dt=1e-4

fig, ax = plt.subplots()

ax.plot(taus, Gts, label=It.lens.p_phys['name'])

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$G(\tau)=dI(\tau)/d\tau/2\pi$')
ax.set_xscale('log')
ax.legend(loc='best');

## Analytic SIS

New implementation of the analytic formula for the SIS. This time I am using a proper implementation of the elliptic integrals (from GSL). The computation now is extremely fast.

In [ ]:
y = 0.3
p_prec = {'eval_mode' : 'exact'}

It_SIS = time_domain_c.It_AnalyticSIS_C(y, p_prec)

## -----------------------------------------------

taus = np.geomspace(1e-1, 10, 10000)
Its = It_SIS.eval_It(taus)/2/np.pi

fig, ax = plt.subplots()

ax.plot(taus, Its, label='SIS')

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.set_xscale('log')
ax.legend(loc='best');

And the Green function is

In [ ]:
y = 0.3
p_prec = {'eval_mode' : 'exact'}

It_SIS = time_domain_c.It_AnalyticSIS_C(y, p_prec)

## -----------------------------------------------

taus = np.geomspace(1e-1, 10, 10000)
Gts = It_SIS.eval_Gt(taus)

fig, ax = plt.subplots()

ax.plot(taus, Gts, label='SIS')

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$G(\tau)=dI(\tau)/d\tau/2\pi$')
ax.set_xscale('log')
ax.set_ylim([-15, 15])
ax.axhline(y=0, c='grey', alpha=0.5)
ax.legend(loc='best');

## Single integral

Strong lensing examples for the CIS and NFW:

In [ ]:
Psi = lenses.Psi_CIS()

y = 0.3
p_prec = {'eval_mode':'exact'}

It = time_domain_c.It_SingleIntegral_C(Psi, y, p_prec)

## ---------------------------------------------------------

taus = np.geomspace(1e-1, 10, 1000)

fig, ax = plt.subplots()

ax.plot(taus, It.eval_It(taus)/2/np.pi, label=It.lens.p_phys['name'])

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.set_xscale('log')
ax.legend(loc='best');

The step function is built-in in the exact mode:

In [ ]:
Psi = lenses.Psi_NFW({'xs':0.1})

y = 0.3
p_prec = {'eval_mode':'exact'}

It = time_domain_c.It_SingleIntegral_C(Psi, y, p_prec)

## ---------------------------------------------------------

taus = np.linspace(-0.1, 2, 1000)

fig, ax = plt.subplots()

ax.plot(taus, It.eval_It(taus)/2/np.pi, label=It.lens.p_phys['name'])

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.legend(loc='best');

## Area integral

This method is slow and noisy compared with the others, and its current implementation is very simple. However, it has been included to cross-check the results of the other methods, since it works equally bad for all lenses. 

In [ ]:
Psi = lenses.Psi_CIS({'rc':0.1})

y = 0.2
p_prec = {'n_rho' : 20000,\
          'tmax' : 3,\
          'Nt' : 500}

It = time_domain_c.It_AreaIntegral_C(Psi, y, p_prec)

In [ ]:
It.display_info()

In [ ]:
fig, ax = plt.subplots()

ax.plot(It.t_grid, It.It_grid/2/np.pi, label=It.lens.p_phys['name'])

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.legend(loc='best');

## Numerical lenses

When the lens is not implemented in C, the code will try to precompute it in a grid, using the Python version, and then read it in C. There is a performance overhead, that can be mitigated reducing the size of the grid, but this feature will (hopefully) increase the flexibility of the code.

In [ ]:
Psi = lenses.Psi_SIS()
Psi.p_phys['name'] = 'new SIS'

y = 1.4
p_prec = {'eval_mode' : 'interpolate'}
It = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

For now, only axisymmetric lenses are allowed.

## Getting contours

The `SingleContour`, `SingleIntegral` and `MultiContour` classes contain a method to output the contours of constant time delay.

In [ ]:
# auxiliary function to map times to colors
def color_scheme(ts, cmap='rainbow', log_norm=False):
        if log_norm is True:
            log_tmin = np.log10(ts[0])
            log_tmax = np.log10(ts[-1])
            normalization = (np.log10(ts) - log_tmin)/(log_tmax - log_tmin)
        else:
            normalization = (ts - ts[0])/(ts[-1] - ts[0])

        cm = plt.colormaps.get_cmap(cmap)
        colors = cm(normalization)
    
        return colors

### Single contour

We can obtain an array of contours for a given array of $\tau$ values. We must also specify the number of points in the contour. Each point is obtained integrating the differential equation (not interpolating).

In [ ]:
y = 1.05
Psi = lenses.Psi_SIS()
It_std = time_domain_c.It_SingleContour_C(Psi, y, {'method':'standard', 'eval_mode':'exact'})
        
n_points = 100
n_contours = 30
ts = np.geomspace(1e-3, 10, n_contours)
cnts_std = It_std.get_contour(ts, n_points=n_points)

## ------------------------------------------------------------

fig, ax = plt.subplots()
for cnt, color in zip(cnts_std, color_scheme(ts)):
    ax.plot(cnt['x1'], cnt['x2'], c=color)

ax.set_aspect('equal')
ax.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It_std.y))
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$');

If the number of points is not specified (or set to be smaller than 1), then the contour contains the accepted points in the computation of $I(\tau)$. This information is useful for debugging. In this method, we can also obtain the contours in terms of $(R, \alpha)$, i.e. the polar coordinates actually used in the integration. Below we also compare with the 'robust' method, where the contours are computed as a parametric representation $R(\sigma)$ and $\alpha(\sigma)$ (the variable $\sigma$ is also stored in the contour dictionary).

In [ ]:
y = 1.05
Psi = lenses.Psi_SIS()
It_std = time_domain_c.It_SingleContour_C(Psi, y, {'method':'standard', 'eval_mode':'exact'})
It_rob = time_domain_c.It_SingleContour_C(Psi, y, {'method':'robust', 'eval_mode':'exact'})

n_contours = 30
ts = np.geomspace(1e-3, 10, n_contours)
cnts_std = It_std.get_contour(ts)
cnts_rob = It_rob.get_contour(ts)

## -----------------------------------------------------

fig, (ax, ax2) = plt.subplots(ncols=2, figsize=(8, 4), gridspec_kw={'wspace':0.3})
for cnt1, cnt2, color in zip(cnts_std, cnts_rob, color_scheme(ts)):
    ax.plot(cnt1['x1'], cnt1['x2'], c=color)
    ax.plot(cnt2['x1'], cnt2['x2'], c=color, ls='--')

    ax2.plot(cnt1['alpha']/np.pi/2., cnt1['R'], c=color)
    ax2.plot(cnt2['alpha']/np.pi/2., cnt2['R'], c=color, ls='--')
    
ax.set_aspect('equal')
ax.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It_std.y))
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')

ax2.set_box_aspect(1)
ax2.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It_std.y))
ax2.set_xlabel('$\\alpha/2\\pi$')
ax2.set_ylabel('$R$');

### Single integral

Even though in this algorithm the contours are irrelevant, they can also be computed. For debugging purposes, the relevant quantities in this method are the minimum an maximum values of $x_1$ at $x_2=0$ for a given contour. These set the upper and lower limits for the radial integral.

In [ ]:
y = 0.1
Psi = lenses.Psi_PointLens()
It_int = time_domain_c.It_SingleIntegral_C(Psi, y, {'eval_mode':'exact'})

n_points = 1000
n_contours = 30
ts = np.geomspace(1e-3, 1, n_contours)
cnts_int = It_int.get_contour(ts, n_points=n_points)

## ------------------------------------------------------------

fig, ax = plt.subplots()
for cnt, color in zip(cnts_int, color_scheme(ts)):
    for x1, x2 in zip(cnt['x1'], cnt['x2']):
        ax.plot(x1, x2, c=color)

ax.set_aspect('equal')
ax.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It_int.y))
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$');

In [ ]:
y = 0.1
Psi = lenses.Psi_SIS()
It_int = time_domain_c.It_SingleIntegral_C(Psi, y, {'eval_mode':'exact'})

n_points = 1000
n_contours = 30
ts = np.geomspace(1e-3, 1, n_contours)
cnts_int = It_int.get_contour(ts, n_points=n_points)

## ------------------------------------------------------------

fig, ax = plt.subplots()
for cnt, color in zip(cnts_int, color_scheme(ts)):
    for x1, x2 in zip(cnt['x1'], cnt['x2']):
        ax.plot(x1, x2, c=color)

ax.set_aspect('equal')
ax.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It_int.y))
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$');

### Multicontour

In [ ]:
def benchmark_lens():
    # benchmark lens for 2d SL    
    xs = [[0.3, 0], [-0.6, 0.3], [0.3, -0.3], [0, 0]]
    psi0 = 1./len(xs)
    rc = 0.05
    Psis = [lenses.Psi_offcenterCIS({'psi0':psi0, 'rc':rc, 'xc1':x[0], 'xc2':x[1]}) for x in xs]
    Psi = lenses.CombinedLens({'lenses':Psis})
    return Psi

## ------------------------------------------------------------

y = 0.05
Psi = benchmark_lens()
It = time_domain_c.It_MultiContour_C(Psi, y, {'eval_mode':'exact'})

n_points = 2000
n_contours = 60
ts = np.linspace(1e-2, 0.15, n_contours)
cnts = It.get_contour(ts, n_points=n_points)

## ------------------------------------------------------------

fig, ax = plt.subplots()
for cnt, color in zip(cnts, color_scheme(ts)):
    for x1, x2 in zip(cnt['x1'], cnt['x2']):
        ax.plot(x1, x2, c=color)

ax.set_aspect('equal')
ax.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It.y))
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$');

# Frequency domain

## General purpose F(w)

The new object for the computation of the amplification factor `Fw_FFT_C` is a general purpose algorithm. It uses the new regularization scheme, trying to improve the precision (and speed) of the computation when information about the asymptotic behaviour of the lens is available. At the moment it can handle power-law tails, but not yet logarithmic tails (even though it is not coded, one can still improve the behaviour with log tails introducing a power-law tail that is "close")

The algorithm first computes the Fourier transform of the regularized $I(\tau)$ and then add to it the analytic FT of the singular contribution. It is always a good idea to plot `Fw_reg` separately from `Fw`, since this is the one that is actually computed numerically.

In [ ]:
y = 1.3
It = time_domain_c.It_AnalyticSIS_C(y)
Fw = freq_domain_c.Fw_FFT_C(It)

## ----------------------------------------

ws = Fw.w_grid

fig, ax = plt.subplots()
ax.plot(ws, np.abs(Fw.eval_Fw(ws)), label='full = sing + reg', alpha=0.5)
ax.plot(ws, np.abs(Fw.eval_Fw_sing(ws)), label='sing')
ax.plot(ws, np.abs(Fw.eval_Fw_reg(ws)), label='reg')
ax.set_xscale('log')
ax.set_xlabel('$w$')
ax.set_ylabel('$|F(w)|$')
ax.legend()
ax.set_title("y=%g" % y)
ax.grid()

In [ ]:
y = 0.3
It = time_domain_c.It_AnalyticSIS_C(y)
Fw = freq_domain_c.Fw_FFT_C(It)

## ----------------------------------------

ws = Fw.w_grid

fig, ax = plt.subplots()
ax.plot(ws, np.abs(Fw.eval_Fw(ws)), label='full = sing + reg', alpha=0.5)
ax.plot(ws, np.abs(Fw.eval_Fw_sing(ws)), label='sing')
ax.plot(ws, np.abs(Fw.eval_Fw_reg(ws)), label='reg')
ax.set_xscale('log')
ax.set_xlabel('$w$')
ax.set_ylabel('$|F(w)|$')
ax.set_title("y=%g" % y)
ax.legend()
ax.grid()

It is also advisable to plot the regularized version of $I(\tau)$, in order to see what is the object actually dealing with. 

In [ ]:
y = 0.3
It = time_domain_c.It_AnalyticSIS_C(y)
Fw = freq_domain_c.Fw_FFT_C(It)

## ----------------------------------------

ts = np.geomspace(1e-1, 1e2, 1000)

fig, ax = plt.subplots()
ax.plot(ts, Fw.It.eval_It(ts), label='full = sing + reg', alpha=0.5)
ax.plot(ts, Fw.eval_It_sing(ts), label='sing')
ax.plot(ts, Fw.eval_It_reg(ts), label='reg')
ax.set_xscale('log')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)$')
ax.set_title("y=%g" % y)
ax.legend()
ax.grid()

## Pseudo-analytic SIS

This is the pseudo-analytic computation of the amplification factor for the SIS. The implementation is very fast in the SL regime but struggles a bit more in the WL regime. It should not be a problem since it is in this regime where the numerical computation shines.

In [ ]:
y = 1.3
Fw = freq_domain_c.Fw_SemiAnalyticSIS_C(y)

## ----------------------------------------

ws = np.geomspace(1e-2, 1e2, 1000)

fig, ax = plt.subplots()
ax.plot(ws, np.abs(Fw.eval_Fw(ws)), label='full = sing + reg', alpha=0.5)
ax.plot(ws, np.abs(Fw.eval_Fw_sing(ws)), label='sing')
ax.plot(ws, np.abs(Fw.eval_Fw_reg(ws)), label='reg')
ax.set_xscale('log')
ax.set_xlabel('$w$')
ax.set_ylabel('$|F(w)|$')
ax.legend()
ax.set_title("y=%g" % y)
ax.grid()

In [ ]:
y = 0.3
Fw = freq_domain_c.Fw_SemiAnalyticSIS_C(y)

## ----------------------------------------

ws = np.geomspace(1e-2, 1e2, 1000)

fig, ax = plt.subplots()
ax.plot(ws, np.abs(Fw.eval_Fw(ws)), label='full = sing + reg', alpha=0.5)
ax.plot(ws, np.abs(Fw.eval_Fw_sing(ws)), label='sing')
ax.plot(ws, np.abs(Fw.eval_Fw_reg(ws)), label='reg')
ax.set_xscale('log')
ax.set_xlabel('$w$')
ax.set_ylabel('$|F(w)|$')
ax.set_title("y=%g" % y)
ax.legend()
ax.grid()

In [ ]:
y = 0.3
Fw = freq_domain_c.Fw_SemiAnalyticSIS_C(y)

## ----------------------------------------

ts = np.geomspace(1e-1, 1e2, 1000)

fig, ax = plt.subplots()
ax.plot(ts, Fw.It.eval_It(ts), label='full = sing + reg', alpha=0.5)
ax.plot(ts, Fw.eval_It_sing(ts), label='sing')
ax.plot(ts, Fw.eval_It_reg(ts), label='reg')
ax.set_xscale('log')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)$')
ax.set_title("y=%g" % y)
ax.legend()
ax.grid()

# Miscellanea

## Internal information and debugging

All the objects defined in the Python modules share a common structure. They only take as parameters either another object, a `p_phys` or a `p_prec` dictionary. This make it easy to display the internal information of each object. All of them ship with a `display_info()` method, that is also a good remainder of the allowed input parameters.

In [ ]:
Psi = lenses.Psi_SIS()
Psi.display_info()

In [ ]:
It = time_domain_c.It_SingleContour_C(Psi, y=1.2)
It.display_info()

In [ ]:
Fw = freq_domain_c.Fw_FFT_C(It)
Fw.display_info()

The time-domain integral can also show the information about the images found

In [ ]:
It = time_domain_c.It_SingleContour_C(Psi, y=1.2)
It.display_images()

In [ ]:
It = time_domain_c.It_SingleIntegral_C(Psi, y=0.2)
It.display_images()

Another feature, especially useful for debugging, is the ability to print the tree of calls needed to replicate a given object:

In [ ]:
print(Psi)

In [ ]:
print(It)

In [ ]:
print(Fw)

In this simple example it is pretty straightforward, but this feature is useful for debugging the code when the amplification factor has been generated in the middle of a more complicated script, e.g. when we use a random distribution of lenses or a lens generated by a different piece of code.

### C precision parameters

Most of the precision parameters in the C code can be accessed from Python. After defining the main object, the current precision parameters can be visualized with the `display_Cprec` method (including options for non-numerical parameters)

In [ ]:
It = time_domain_c.It_AnalyticSIS_C(y=1.2)
#It.display_Cprec()   # long output

These parameters can be modified through an entry in `p_prec`

In [ ]:
C_prec = {'ro_issameCP_dist' : 2e-5, \
          'sc_intContourStd' : {'type' : 'rkf45', 'epsabs' : 1e-6}}
Psi = lenses.Psi_SIS()
It = time_domain_c.It_SingleContour_C(Psi, y=1.2, p_prec={'C_prec' : C_prec})

If needed, the precision parameters of the current session can also be retrieved in dictionary form

In [ ]:
full_C_prec = It.get_Cprec()

## A more complex example: $n$ SISs

First, we define a couple of functions that will prove handy

In [ ]:
def distribute_centers(n, R=1, seed=None):
    
    if(seed is None):
        seed = np.random.randint(0, 1e7)
        print("Seed:", seed)    
    np.random.seed(seed)
    
    Rs = R*np.sqrt(np.random.random_sample(n))
    ths = 2*np.pi*np.random.random_sample(n)
    
    x1 = Rs*np.cos(ths)
    x2 = Rs*np.sin(ths)
    
    x1CM = np.sum(x1)/n
    x2CM = np.sum(x2)/n
    
    x1 -= x1CM
    x2 -= x2CM
    
    return x1, x2

def create_lens(n, R, seed=None):
    psi0 = 1./n
    xc1s, xc2s = distribute_centers(n, R, seed=seed)
    
    p_phys = {'lenses' : []}
    for xc1, xc2 in zip(xc1s, xc2s):
        sub_p_phys = {'psi0' : psi0, 'xc1' : xc1, 'xc2' : xc2}
        Psi = lenses.Psi_offcenterSIS(sub_p_phys)
        p_phys['lenses'].append(Psi)

    return lenses.CombinedLens(p_phys)

def plot_It(Psi, y, SIS=False, CIS=False, rc=0.1):
    p_prec = {'eval_mode' : 'exact'}
    It = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

    taus = np.geomspace(1e-1, 50, 1000)

    fig, ax = plt.subplots()
    ax.plot(taus, It.eval_It(taus)/2/np.pi, label='%d SIS' % len(Psi.p_phys['lenses']))
    ax.set_xscale('log')
    
    It_SIS = None
    if SIS is True:
        It_SIS = time_domain_c.It_AnalyticSIS_C(y, p_prec)
        ax.plot(taus, It_SIS.eval_It(taus)/2/np.pi, label='SIS')
    
    It_CIS = None
    if CIS is True:
        Psi_CIS = lenses.Psi_CIS({'psi0':1, 'rc':rc})
        It_CIS = time_domain_c.It_SingleContour_C(Psi_CIS, y, p_prec)
        ax.plot(taus, It_CIS.eval_It(taus)/2/np.pi, label='CIS')
    
    ax.set_xlabel(r'$\tau$')
    ax.set_ylabel(r'$I(\tau)/2\pi$')
    ax.set_xlim([taus[0], taus[-1]])
    ax.legend(loc='best')
    
def plot_Gt(Psi, y, SIS=False, CIS=False, rc=0.1):
    p_prec = {'eval_mode' : 'exact'}
    It = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

    dt = 1e-3
    taus = np.geomspace(1e-1, 50, 1000)
    
    fig, ax = plt.subplots()
    ax.plot(taus, It.eval_Gt(taus, dt=dt), label='%d SIS' % len(Psi.p_phys['lenses']), c='C0')
    ax.set_xscale('log')
    
    It_SIS = None
    if SIS is True:
        It_SIS = time_domain_c.It_AnalyticSIS_C(y, p_prec)
        ax.plot(taus, It_SIS.eval_Gt(taus, dt=dt), label='SIS', c='C1')
    
    It_CIS = None
    if CIS is True:
        Psi_CIS = lenses.Psi_CIS({'psi0':1, 'rc':rc})
        It_CIS = time_domain_c.It_SingleContour_C(Psi_CIS, y, p_prec)
        ax.plot(taus, It_CIS.eval_Gt(taus, dt=dt), label='CIS', c='C2')
    
    ax.set_xlabel(r'$\tau$')
    ax.set_ylabel(r'$G(\tau)$')
    ax.set_xlim([taus[0], taus[-1]])
    ax.legend(loc='best')

Now, we will create a set of (`n_sublenses`) SIS randomly distributed inside a sphere of radius (`R`). The center of mass position and the total lens mass are fixed.

In [ ]:
n_sublenses = 3
R = 1

Psi = create_lens(n=n_sublenses, R=R, seed=None)

We can compare this composed lens with a SIS or a CIS of the same mass (situated at the center of mass of the lens distribution)

In [ ]:
plot_It(Psi, y=1.2, SIS=True, CIS=True)

In [ ]:
plot_Gt(Psi, y=1.2, SIS=False, CIS=True)

We can also compare the effects in the amplification factor. First we must compute the time-domain integral in a wide temporal range, as usual. (Note: the plot above was computed using the `p_prec['eval_method']='exact'` option to compute it at any tau requested, now we need a grid so we must use `'interpolate'`, which is the default)

In [ ]:
y = 1.2
p_prec = {'tmin':1e-2, 'tmax':1e8, 'Nt':5000, 'sampling':'log'}

It_MultiSIS = time_domain_c.It_SingleContour_C(Psi, y, p_prec)

It_SIS = time_domain_c.It_AnalyticSIS_C(y, p_prec)

Psi_CIS = lenses.Psi_CIS({'psi0':1, 'rc':0.1})
It_CIS = time_domain_c.It_SingleContour_C(Psi_CIS, y, p_prec)

Next, we compute the amplification factors

In [ ]:
Fw_MultiSIS = freq_domain_c.Fw_FFT_C(It_MultiSIS)
Fw_SIS = freq_domain_c.Fw_FFT_C(It_SIS)
Fw_CIS = freq_domain_c.Fw_FFT_C(It_CIS)

and plot them

In [ ]:
fig, ax = plt.subplots()

ws = Fw_SIS.w_grid

ws = np.geomspace(1e-2, 1e2, 5000)

ax.plot(ws, np.abs(Fw_MultiSIS(ws)), zorder=10, label='%d SIS' % len(Psi.p_phys['lenses']))
ax.plot(ws, np.abs(Fw_SIS(ws)), alpha=0.5, label='SIS')
ax.plot(ws, np.abs(Fw_CIS(ws)), alpha=0.5, label='CIS')

ax.set_xlim([ws[0], ws[-1]])
ax.set_xscale('log')
ax.set_ylabel('$|F(w)|$')
ax.set_xlabel('$w$')
ax.legend(loc='best');

## Multicontour plots

### Debugging 

#### Definitions

In [ ]:
def benchmark_lens():
    # benchmark lens for 2d SL    
    xs = [[0.3, 0], [-0.6, 0.3], [0.3, -0.3], [0, 0]]
    psi0 = 1./len(xs)
    rc = 0.05
    Psis = [lenses.Psi_offcenterCIS({'psi0':psi0, 'rc':rc, 'xc1':x[0], 'xc2':x[1]}) for x in xs]
    Psi = lenses.CombinedLens({'lenses':Psis})
    return Psi

def plot_saddle_cnt(It, ax, dx=1e-3):
    saddles = [p_crit for p_crit in It.p_crits if p_crit['type'] == 'saddle']
    
    xmax = 0
    for s in saddles:        
        x1 = s['x1']
        x2 = s['x2']
        d11 = Psi.ddpsi_ddx1(x1, x2)
        d22 = Psi.ddpsi_ddx2(x1, x2)
        d12 = Psi.ddpsi_dx1dx2(x1, x2)
        
        kappa  = 0.5*(d11 + d22)
        gamma1 = 0.5*(d11 - d22)
        gamma2 = d12
        gamma = np.sqrt(gamma1**2 + gamma2**2)
        
        C = (gamma1 + gamma)/(gamma2 + 1e-12)
        
        l1 = 1 + kappa + gamma
        l2 = 1 + kappa - gamma
        
        v1 = np.array([C, 1])
        v2 = np.array([1, -C])
        
        x0 = np.array([x1, x2])
        x_a = x0 + v1*dx
        x_b = x0 - v1*dx
        x_c = x0 + v2*dx
        x_d = x0 - v2*dx
        
        for xs in [x_a, x_b, x_c, x_d]:
            x1, x2 = xs
            cnt = It.get_contour_x1x2(x1, x2, sigmaf=300, n_points=1000)
            ax.plot(cnt['x1'], cnt['x2'], lw=0.6, c='black')
            ax.plot(cnt['x1'][0], cnt['x2'][0], marker='o', ms=3, ls='', c='black')
            
            xmax = max(np.max(np.abs(cnt['x1'])), np.max(np.abs(cnt['x2'])), xmax)
            
    return xmax, ax
    
def plot_centers(It, ax):
    for c in It.p_centers:
        x1, x2 = c['x10'], c['x20']
        if c['is_init_birthdeath'] == 1:
            color = 'green'
            R, a = c['R_max'], c['alpha_out']
        else:
            color = 'red'
            R, a = 0.1, c['alpha_out']
            
        ax.plot(x1, x2, marker='o', ms=4, ls='', c=color)
        
        dx1, dx2 = R*np.cos(a), R*np.sin(a)
        ax.arrow(x1, x2, dx1, dx2,
                 ec = color,
                 fc = color,
                 length_includes_head = True)
                 
    return ax

def plot_debugMulti(Psi, y, N=200, source=False, images=False, lenses=False, saddles=False, centers=False, fig=None, ax=None):
    # compute the images and centers only
    It = time_domain_c.It_MultiContour_C(Psi, y, {'eval_mode' : 'exact'})
    
    # init figure
    if (fig is None) and (ax is None):
        fig, ax = plt.subplots()
    
    # find the plot limits
    xmax_images = 1.3*np.abs(np.array([[p['x1'], p['x2']] for p in It.p_crits]).flatten()).max()
    xmax = xmax_images
    
    # plot the critical contours
    if saddles is True:
        xmax_saddle, ax = plot_saddle_cnt(It, ax, dx=1e-3)
        xmax = max(xmax, xmax_saddle)
    
    # plot the lens eq
    fig, ax = tools.plot_lensEq_2d(Psi, y, x1min=-xmax, x1max=xmax, Nx=N, fig=fig, ax=ax)
    
    # plot the location of the images
    if images is True:
        for i, p in enumerate(It.p_crits):
            ax.text(p['x1'], p['x2'], "%d" % i, fontsize='small')
    
    # plot the location of the source
    if source is True:
        ax.plot(y, 0, marker='*', ms=5, ls='', c='black')
    
    # plot the location of the sublenses
    if lenses is True:
        try:
            for lens in Psi.p_phys['lenses']:
                x1 = lens.p_phys['xc1']
                x2 = lens.p_phys['xc2']
                ax.plot(x1, x2, marker='x', ms=4, ls='', c='black', alpha=0.5)
        except KeyError:
            ax.plot(0, 0, marker='x', ms=4, ls='', c='black', alpha=0.5)
        
    # plot the centers
    if centers is True:
        ax = plot_centers(It, ax)
        
    return fig, ax, It

#### Plots

In [ ]:
y = 0.015
Psi = benchmark_lens()

# default behaviour : plot the lens equation
fig, ax, It = plot_debugMulti(Psi, y)

In [ ]:
#interesting values
y = 0.015
#y = 0.05
#y = 0.08
#y = 0.2
#y = 0.3

Psi = benchmark_lens()

kwargs = {}
kwargs['N']       = 200   # number of points in the grid for imshow
kwargs['source']  = True  # source (with a star)
kwargs['images']  = True  # images with numbers (sorted with tau min -> max)
kwargs['lenses']  = True  # position of the sublenses (with crosses)
kwargs['saddles'] = True  # critical curves for the saddle points
kwargs['centers'] = True  # centers with the min and max radius (in the direction out)
                          # outer center in red

fig, ax, It = plot_debugMulti(Psi, y, **kwargs)

### Advanced contour plots

#### Definitions

In [ ]:
# benchmark lens for 2d SL
def benchmark_lens():    
    xs = [[0.3, 0], [-0.6, 0.3], [0.3, -0.3], [0, 0]]
    psi0 = 1./len(xs)
    rc = 0.05
    Psis = [lenses.Psi_offcenterCIS({'psi0':psi0, 'rc':rc, 'xc1':x[0], 'xc2':x[1]}) for x in xs]
    Psi = lenses.CombinedLens({'lenses':Psis})
    return Psi

# auxiliary function to map times to colors
def color_scheme(ts, cmap='rainbow', log_norm=False):
        if log_norm is True:
            log_tmin = np.log10(ts[0])
            log_tmax = np.log10(ts[-1])
            normalization = (np.log10(ts) - log_tmin)/(log_tmax - log_tmin)
        else:
            normalization = (ts - ts[0])/(ts[-1] - ts[0])

        cm = plt.colormaps.get_cmap(cmap)
        colors = cm(normalization)
    
        return colors

# convert array into colorized segments
def plot_segments(ax, x, y, color_of_x):
    colors = color_of_x(x)
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    line_segments = matplotlib.collections.LineCollection(segments, colors=colors[:-1])

    ax.add_collection(line_segments)
    ax.autoscale()
    return ax

def plot_contours(Psi, y, tmin=3e-3, tmax=0.3, Nt=60, n_points=1000, \
                  log=False, cmap='rainbow', fcolor='white', plot_all=False):
    It = time_domain_c.It_MultiContour_C(Psi, y, {'eval_mode':'exact'})

    if log is False:
        ts = np.linspace(tmin, tmax, Nt)
    else:
        ts = np.geomspace(tmin, tmax, Nt)
        
    cnts = It.get_contour(ts, n_points=n_points)

    ## ------------------------------------------------------------

    if plot_all is True:
        #fig = plt.figure(layout="constrained")
        fig = plt.figure()
        gs = matplotlib.gridspec.GridSpec(2, 2, figure=fig, wspace=0.05, hspace=0.05)
        ax = fig.add_subplot(gs[:, 0])
        
        ax_It = fig.add_subplot(gs[0, 1])
        ax_Gt = fig.add_subplot(gs[1, 1])
        axs = (ax_It, ax_Gt)

        # populate more densely and plot as colored segments
        if log is False:
            x = np.linspace(tmin, tmax, 20*Nt)
        else:
            x = np.geomspace(tmin, tmax, 20*Nt)

        y = It.eval_It(x)/2/np.pi
        plot_segments(ax_It, x, y, lambda x: color_scheme(x, cmap=cmap))

        y = It.eval_Gt(x)
        plot_segments(ax_Gt, x, y, lambda x: color_scheme(x, cmap=cmap))

        # customize plot
        ax_It.get_xaxis().set_visible(False)
        ax_It.set_ylabel('$I(\\tau)/2\\pi$')
        ax_Gt.set_ylabel('$G(\\tau)$')
        ax_Gt.set_xlabel('$\\tau$')
        
        for a in axs:
            a.yaxis.tick_right()
            a.yaxis.set_label_position("right")
            a.set_facecolor(fcolor)
    else:
        fig, ax = plt.subplots()
        
    for cnt, color in zip(cnts, color_scheme(ts, cmap=cmap)):
        for x1, x2 in zip(cnt['x1'], cnt['x2']):
            ax.plot(x1, x2, c=color)

    ax.set_aspect('equal')
    ax.set_title('%s $(y=%g)$' % (Psi.p_phys['name'], It.y))
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$');
    ax.set_facecolor(fcolor)

    return fig

#### Plots

In [ ]:
y = 0.01
Psi = benchmark_lens()

# default behaviour : plot only some contours
fig = plot_contours(Psi, y)

In [ ]:
y = 0.015
Psi = benchmark_lens()

kwargs = {}
kwargs['tmin'] = 1e-3       # min tau
kwargs['tmax'] = 0.19       # max tau
kwargs['Nt']   = 40         # number of contours (for It -> N=20*Nt)
kwargs['n_points'] = 1000   # number of points in each contour
kwargs['log'] = True        # if True -> log spacing in tau
kwargs['cmap'] = 'rainbow'  # colormap
kwargs['fcolor'] = 'black'  # background color
kwargs['plot_all'] = True   # if True plot It and Gt as well

fig = plot_contours(Psi, y, **kwargs)

## Testing precision

In [ ]:
from glow import wrapper

### Single contour

Benchmark values (RK(8,9) integrator with tol=1e-5, compilation with `-Ofast`)

In [ ]:
y = 1.1
psi0 = 1
x1_min = psi0 + y
x2_min = 0

Psi = lenses.Psi_SIS({'psi0':psi0})

tau_ini = 0.8
taus = np.geomspace(0.1, 100, 1000)

We can compare it with the exact result

In [ ]:
Is = wrapper.pyContour(taus, x1_min, x2_min, y, Psi)
Is2 = wrapper.pyContour(taus, x1_min, x2_min, y, Psi, method='robust')
It_exact = wrapper.pyIt_SIS(taus, y, psi0)

fig, ax = plt.subplots()
ax.plot(taus, np.abs(Is-It_exact)/It_exact, label='std method')
ax.plot(taus, np.abs(Is2-It_exact)/It_exact, label='robust method')
ax.set_xlabel('$\\tau$')
ax.set_ylabel('$|I-I_{exact}|/|I_{exact}|$')
ax.legend()
ax.grid()
ax.set_xscale('log')
ax.set_yscale('log')

### Single integral

Benchmark values: `qags` integrator with rtol=1e-4, atol=0, sub_int_limit=1000, compilation with `-Ofast`)

In [ ]:
y = 1.3

tau_ini = 0.8
taus = np.geomspace(1e-2, 10, 1000)

Psi = lenses.Psi_SIS()

# extract p_crits first
It = time_domain_c.It_SingleIntegral_C(Psi, y, {'eval_mode':'exact'})
p_crits = It.p_crits

We can compare it with the exact result

In [ ]:
Is = wrapper.pySingleIntegral(taus, y, Psi, p_crits, method='direct')
It_exact = wrapper.pyIt_SIS(taus, y, 1)

fig, ax = plt.subplots()
ax.plot(taus, np.abs(Is-It_exact)/It_exact)
ax.grid()
ax.set_xlabel('$\\tau$')
ax.set_ylabel('$|I-I_{exact}|/|I_{exact}|$')
ax.set_xscale('log')
ax.set_yscale('log')

### General F(w)

Weak-lensing precision

In [ ]:
y = 1.3

It_std = time_domain_c.It_AnalyticSIS_C(y)
Fw_std = freq_domain_c.Fw_FFT_C(It_std)

It_high = time_domain_c.It_AnalyticSIS_C(y, {'sampling':'oversampling'})
Fw_high = freq_domain_c.Fw_FFT_C(It_high, p_prec={'N_above_discard':8})

Fw_exact = freq_domain_c.Fw_SemiAnalyticSIS_C(y)

Fws_std  = Fw_std.Fw_grid
Fws_high = Fw_high.Fw_grid
Fws_std_exact  = Fw_exact.eval_Fw(Fw_std.w_grid)
Fws_high_exact = Fw_exact.eval_Fw(Fw_high.w_grid)

fig, ax = plt.subplots()
ax.plot(Fw_std.w_grid, np.abs(Fws_std-Fws_std_exact)/np.abs(Fws_std_exact), label='std prec')
ax.plot(Fw_high.w_grid, np.abs(Fws_high-Fws_high_exact)/np.abs(Fws_high_exact), label='high prec')
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('y=%g' % y)
ax.set_xlabel('$w$')
ax.set_ylabel('$|F-F_{exact}|/|F_{exact}|$')
ax.grid()

Strong-lensing precision

In [ ]:
y = 0.3

It_std = time_domain_c.It_AnalyticSIS_C(y)
Fw_std = freq_domain_c.Fw_FFT_C(It_std)

It_high = time_domain_c.It_AnalyticSIS_C(y, {'sampling':'oversampling'})
Fw_high = freq_domain_c.Fw_FFT_C(It_high, p_prec={'N_above_discard':8})

Fw_exact = freq_domain_c.Fw_SemiAnalyticSIS_C(y)

Fws_std  = Fw_std.Fw_grid
Fws_high = Fw_high.Fw_grid
Fws_std_exact  = Fw_exact.eval_Fw(Fw_std.w_grid)
Fws_high_exact = Fw_exact.eval_Fw(Fw_high.w_grid)

fig, ax = plt.subplots()
ax.plot(Fw_std.w_grid, np.abs(Fws_std-Fws_std_exact)/np.abs(Fws_std_exact), label='std prec')
ax.plot(Fw_high.w_grid, np.abs(Fws_high-Fws_high_exact)/np.abs(Fws_high_exact), label='high prec')
ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('y=%g' % y)
ax.set_xlabel('$w$')
ax.set_ylabel('$|F-F_{exact}|/|F_{exact}|$')
ax.grid()